### ========================================================
### EXAMEN FINAL - RECUPERACIÓN DE INFORMACIÓN CON RAG
### Autor: Christian Perez
### Fecha: 28 de julio 2025
### =======================================================

## 1. Bibliotecas necesarias

In [1]:
import pandas as pd
import json
import nltk
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi
import faiss
from sentence_transformers import SentenceTransformer
import google.generativeai as genai
import numpy as np

# Descargar stopwords si es necesario
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))


C:\Users\Christian\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Carga y preprocesamiento del Corpus (1% del corpus)

In [2]:
# Carga del archivo JSON
data_path = './data/arxiv-metadata-oai-snapshot.json'
records = []
## El 1% del copus es: 2,792,339×0.01=27,923.39

## Se utilizo este codigo para sabaer la longitud del corpus:
##corpus_path = "./data/arxiv-metadata-oai-snapshot.json"
##total_docs = 0
##with open(corpus_path, 'r', encoding='utf-8') as f:
##    for _ in f:
##        total_docs += 1
##print(f"El corpus original contiene {total_docs} documentos.")

with open(data_path, 'r') as file:
    for i, line in enumerate(file):
        if i >= 28000:
            break
        record = json.loads(line)
        records.append(record)

# Crear DataFrame inicial
df = pd.DataFrame(records)
df.head()


,id,submitter,authors,title,comments,journal-ref,doi,report-no,categories,license,abstract,versions,update_date,authors_parsed
0,0704.0001,Pavel Nadolsky,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",Calculation of prompt diphoton production cros...,"37 pages, 15 figures; published version","Phys.Rev.D76:013009,2007",10.1103/PhysRevD.76.013009,ANL-HEP-PR-07-12,hep-ph,None,A fully differential calculation in perturba...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2008-11-26,"[[Balázs, C., ], [Berger, E. L., ], [Nadolsky,..."
1,0704.0002,Louis Theran,Ileana Streinu and Louis Theran,Sparsity-certifying Graph Decompositions,To appear in Graphs and Combinatorics,None,None,None,math.CO cs.CG,http://arxiv.org/licenses/nonexclusive-distrib...,"We describe a new algorithm, the $(k,\ell)$-...","[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2008-12-13,"[[Streinu, Ileana, ], [Theran, Louis, ]]"
2,0704.0003,Hongjun Pan,Hongjun Pan,The evolution of the Earth-Moon system based o...,"23 pages, 3 figures",None,None,None,physics.gen-ph,None,The evolution of Earth-Moon system is descri...,"[{'version': 'v1', 'created': 'Sun, 1 Apr 2007...",2008-01-13,"[[Pan, Hongjun, ]]"
3,0704.0004,David Callan,David Callan,A determinant of Stirling cycle numbers counts...,11 pages,None,None,None,math.CO,None,We show that a determinant of Stirling cycle...,"[{'version': 'v1', 'created': 'Sat, 31 Mar 200...",2007-05-23,"[[Callan, David, ]]"
4,0704.0005,Alberto Torchinsky,Wael Abu-Shammala and Alberto Torchinsky,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,None,"Illinois J. Math. 52 (2008) no.2, 681-689",None,None,math.CA math.FA,None,In this paper we show how to compute the $\L...,"[{'version': 'v1', 'created': 'Mon, 2 Apr 2007...",2013-10-15,"[[Abu-Shammala, Wael, ], [Torchinsky, Alberto, ]]"


In [3]:
# Función de preprocesamiento
def preprocess(text):
    """
    Preprocesa el texto:
    - Convierte a minúsculas
    - Elimina puntuación
    - Tokeniza
    - Elimina stopwords
    """
    if pd.isna(text):
        return ""
    
    text = str(text).lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    tokens = nltk.word_tokenize(text)
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]
    return ' '.join(tokens)
# Aplicar preprocesamiento al título y resumen combinados
df['processed_text'] = (df['title'].fillna('') + ' ' + df['abstract'].fillna('')).apply(preprocess)
# Filtrar documentos vacíos después del preprocesamiento
df = df[df['processed_text'].str.len() > 0].reset_index(drop=True)
print(f"Documentos después de preprocesamiento: {len(df)}")
print("Muestra de datos procesados:")
df[['id', 'title', 'processed_text']].head()

Documentos después de preprocesamiento: 28000
Muestra de datos procesados:


,id,title,processed_text
0,0704.0001,Calculation of prompt diphoton production cros...,calculation prompt diphoton production cross s...
1,0704.0002,Sparsity-certifying Graph Decompositions,sparsitycertifying graph decompositions descri...
2,0704.0003,The evolution of the Earth-Moon system based o...,evolution earthmoon system based dark matter f...
3,0704.0004,A determinant of Stirling cycle numbers counts...,determinant stirling cycle numbers counts unla...
4,0704.0005,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,dyadic lambdaalpha lambdaalpha paper show comp...


## 3. Indexación TF-IDF

In [4]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['processed_text'])
print(f"Matriz TF-IDF creada: {tfidf_matrix.shape}")

Matriz TF-IDF creada: (28000, 96974)


## 4. Indexación BM25

In [5]:
tokenized_corpus = [doc.split() for doc in df['processed_text']]
bm25 = BM25Okapi(tokenized_corpus)

## 5. Indexación vectorial con FAISS

In [6]:
# Embeddings con sentence-transformers
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
# Generar embeddings en lotes para manejar memoria
batch_size = 32
embeddings_list = []
for i in range(0, len(df), batch_size):
    batch_texts = df['processed_text'].iloc[i:i+batch_size].tolist()
    batch_embeddings = embedding_model.encode(batch_texts, show_progress_bar=False)
    embeddings_list.append(batch_embeddings)
    if (i // batch_size) % 100 == 0:
        print(f"Procesados {i + len(batch_texts)} documentos")
embeddings = np.vstack(embeddings_list)
print(f"Embeddings generados: {embeddings.shape}")
# Crear índice FAISS
dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings.astype('float32'))
print(f"Índice FAISS creado con {index.ntotal} vectores")

Procesados 32 documentos
Procesados 3232 documentos
Procesados 6432 documentos
Procesados 9632 documentos
Procesados 12832 documentos
Procesados 16032 documentos
Procesados 19232 documentos
Procesados 22432 documentos
Procesados 25632 documentos
Embeddings generados: (28000, 384)
Índice FAISS creado con 28000 vectores


## 6. Funciones para búsqueda

In [7]:
def search_tfidf(query, top_k=10):
    """Búsqueda usando TF-IDF"""
    query_processed = preprocess(query)
    if not query_processed:
        return pd.DataFrame(columns=['id', 'title', 'abstract'])
    query_vec = tfidf_vectorizer.transform([query_processed])
    scores = (tfidf_matrix @ query_vec.T).toarray().flatten()
    top_indices = scores.argsort()[-top_k:][::-1]
    # Filtrar resultados con score > 0
    valid_indices = [i for i in top_indices if scores[i] > 0]
    return df.iloc[valid_indices][['id', 'title', 'abstract']]

In [8]:
def search_bm25(query, top_k=10):
    """Búsqueda usando BM25"""
    query_processed = preprocess(query)
    if not query_processed:
        return pd.DataFrame(columns=['id', 'title', 'abstract'])
    tokenized_query = query_processed.split()
    scores = bm25.get_scores(tokenized_query)
    top_indices = scores.argsort()[-top_k:][::-1]
    # Filtrar resultados con score > 0
    valid_indices = [i for i in top_indices if scores[i] > 0]
    return df.iloc[valid_indices][['id', 'title', 'abstract']]

In [9]:
def search_faiss(query, top_k=10):
    """Búsqueda usando FAISS con embeddings"""
    query_processed = preprocess(query)
    if not query_processed:
        return pd.DataFrame(columns=['id', 'title', 'abstract'])
    query_emb = embedding_model.encode([query_processed])
    distances, indices = index.search(query_emb.astype('float32'), top_k)
    # Filtrar resultados válidos
    valid_indices = [idx for idx in indices[0] if idx != -1]
    return df.iloc[valid_indices][['id', 'title', 'abstract']]


## 7. Módulo RAG

In [ ]:
import google.generativeai as genai
# Configurar clave API para Gemini
genai.configure(api_key='API_KEY') 
# Cliente Gemini
gemini_client = genai

In [11]:
def generar_contexto_faiss(query, top_k=3):
    """Genera contexto usando los mejores documentos de FAISS"""
    documentos = search_faiss(query, top_k)
    contexto = ""
    docs_usados = []

    for _, doc in documentos.iterrows():
        contexto += f"Título: {doc['title']}\n"
        contexto += f"Resumen: {doc['abstract']}\n"
        contexto += f"ID: {doc['id']}\n"
        contexto += "-"*50 + "\n"
        
        # Guardar documento para evaluación
        docs_usados.append({
            'id': doc['id'],
            'title': doc['title'],
            'abstract': doc['abstract']
        })

    return contexto, docs_usados

In [20]:
def generar_respuesta_rag(query):
    """Genera respuesta usando RAG con Gemini"""
    context, docs_usados = generar_contexto_faiss(query)

    prompt = f"""
Eres un experto en Recuperación de Información científica. Basado en el siguiente contexto de artículos científicos:

{context}

Responde esta pregunta de manera natural y completa: {query}

Instrucciones:
1. Proporciona una respuesta bien estructurada y concisa basada en el contexto.
2. Menciona los artículos relevantes cuando sea apropiado usando sus IDs.
3. Sintetiza la información de múltiples documentos si es necesario.
4. Si un concepto aparece en múltiples documentos, integra la información.
5. Basa tu respuesta únicamente en el contexto proporcionado.
6. Si no hay información suficiente, indica "No existe información suficiente para responder a la pregunta".
7. Si la respuesta que va a ser mostrada al usuario está en inglés, tradúcela al español antes de mostrarla.
"""

    try:
        model_gemini = gemini_client.GenerativeModel('gemini-2.5-flash')
        response = model_gemini.generate_content(prompt)
        answer = response.text
    except Exception as e:
        answer = f"Error al generar respuesta: {str(e)}"

    return answer, docs_usados

In [21]:
# Consulta al RAG
consulta = "machine learning for particle physics"
# Generar respuesta RAG
generar_respuesta_rag(consulta)

('En el contexto proporcionado, el aprendizaje automático (machine learning) en física de partículas se aborda a través del uso de redes neuronales.\n\nEspecíficamente, las redes neuronales se emplean ampliamente en problemas de clasificación dentro de la investigación en física de partículas (ID: 0707.0930). Un ejemplo de su aplicación es la discriminación de señal/fondo en la búsqueda de leptoquarks de segunda generación en el Tevatron (ID: 0707.0930).\n\nEl aprendizaje bayesiano de redes neuronales se presenta como un método que puede ofrecer resultados más óptimos y robustos en comparación con los métodos de aprendizaje convencionales (ID: 0707.0930). Se realiza una comparación entre el entrenamiento convencional de redes neuronales de alimentación directa (feedforward) y aquellas entrenadas con métodos bayesianos (ID: 0707.0930).',
 [{'id': '0708.1161',
   'title': 'A threshold-improved narrow-width approximation for BSM physics',
   'abstract': '  A modified narrow-width approxim

## 8. Evaluación de resultados

In [23]:
print("EVALUACIÓN COMPARATIVA")
print("="*100)

# Leer queries desde el archivo
with open("queries.txt", "r", encoding="utf-8") as f:
    queries = [line.strip() for line in f if line.strip()]

print("Consultas cargadas desde queries.txt:")
for i, q in enumerate(queries, 1):
    print(f"{i}. {q}")

EVALUACIÓN COMPARATIVA
Consultas cargadas desde queries.txt:
1. superconductivity in high temperature materials
2. applications of reinforcement learning
3. gravitational waves detection methods
4. graph neural networks in chemistry
5. bayesian inference in astrophysics


In [25]:
# Estructura para guardar resultados

resultados = []
for query_idx, query in enumerate(queries, 1):
    print(f"{'='*60}")
    print(f"PROCESANDO QUERY {query_idx}/5: {query}")

    # Recuperar top-10 para cada modelo
    tfidf_docs = search_tfidf(query, top_k=10)
    bm25_docs = search_bm25(query, top_k=10)
    faiss_docs = search_faiss(query, top_k=10)

    # IDs de los documentos
    tfidf_ids = tfidf_docs['id'].tolist()
    bm25_ids = bm25_docs['id'].tolist()  
    faiss_ids = faiss_docs['id'].tolist()
    print(f"Resultados obtenidos - TF-IDF: {len(tfidf_ids)}, BM25: {len(bm25_ids)}, FAISS: {len(faiss_ids)}")
    
    # Documentos en común entre modelos
    comunes_tfidf_bm25 = set(tfidf_ids) & set(bm25_ids)
    comunes_tfidf_faiss = set(tfidf_ids) & set(faiss_ids)
    comunes_bm25_faiss = set(bm25_ids) & set(faiss_ids)
    comunes_todos = set(tfidf_ids) & set(bm25_ids) & set(faiss_ids)
    
    # Ejecutar módulo RAG
    respuesta_rag, docs_usados_rag = generar_respuesta_rag(query)
    ids_rag = [d['id'] for d in docs_usados_rag]
    
    # Contar coincidencias de documentos RAG con cada modelo
    ids_en_rag = {
        "en_tfidf": sum([i in tfidf_ids for i in ids_rag]),
        "en_bm25": sum([i in bm25_ids for i in ids_rag]),
        "en_faiss": sum([i in faiss_ids for i in ids_rag])
    }

PROCESANDO QUERY 1/5: superconductivity in high temperature materials
Resultados obtenidos - TF-IDF: 10, BM25: 10, FAISS: 10
PROCESANDO QUERY 2/5: applications of reinforcement learning
Resultados obtenidos - TF-IDF: 10, BM25: 10, FAISS: 10
PROCESANDO QUERY 3/5: gravitational waves detection methods
Resultados obtenidos - TF-IDF: 10, BM25: 10, FAISS: 10
PROCESANDO QUERY 4/5: graph neural networks in chemistry
Resultados obtenidos - TF-IDF: 10, BM25: 10, FAISS: 10
PROCESANDO QUERY 5/5: bayesian inference in astrophysics
Resultados obtenidos - TF-IDF: 10, BM25: 10, FAISS: 10


In [26]:
# Almacenar resultados
resultado = {
        "Query": query,
        "TFIDF_top3": tfidf_ids[:3],
        "BM25_top3": bm25_ids[:3],
        "FAISS_top3": faiss_ids[:3],
        "TFIDF_all": tfidf_ids,
        "BM25_all": bm25_ids,
        "FAISS_all": faiss_ids,
        "Comunes_TFIDF_BM25": len(comunes_tfidf_bm25),
        "Comunes_TFIDF_FAISS": len(comunes_tfidf_faiss),
        "Comunes_BM25_FAISS": len(comunes_bm25_faiss),
        "Comunes_todos": len(comunes_todos),
        "Docs_RAG": ids_rag,
        "RAG_en_TFIDF": ids_en_rag["en_tfidf"],
        "RAG_en_BM25": ids_en_rag["en_bm25"],
        "RAG_en_FAISS": ids_en_rag["en_faiss"],
        "Respuesta_RAG": respuesta_rag,
        "Docs_usados_RAG": docs_usados_rag
    }
    
resultados.append(resultado)
    
print(f"Query completada. Documentos en común: {len(comunes_todos)}")

# Convertir a DataFrame para análisis
df_resultados = pd.DataFrame(resultados)

Query completada. Documentos en común: 1


In [17]:
df_resultados 

,Query,TFIDF_top3,BM25_top3,FAISS_top3,TFIDF_all,BM25_all,FAISS_all,Comunes_TFIDF_BM25,Comunes_TFIDF_FAISS,Comunes_BM25_FAISS,Comunes_todos,Docs_RAG,RAG_en_TFIDF,RAG_en_BM25,RAG_en_FAISS,Respuesta_RAG,Docs_usados_RAG
0,bayesian inference in astrophysics,"[0708.1362, 0709.1504, 0706.1767]","[0707.0930, 0710.0845, 0707.3969]","[0707.1611, 0707.3962, 0709.0970]","[0708.1362, 0709.1504, 0706.1767, 0709.3339, 0...","[0707.0930, 0710.0845, 0707.3969, 0707.2429, 0...","[0707.1611, 0707.3962, 0709.0970, 0704.1808, 0...",4,1,1,1,"[0707.1611, 0707.3962, 0709.0970]",0,0,3,La inferencia bayesiana se aplica ampliamente ...,"[{'id': '0707.1611', 'title': 'Probabilistic C..."


In [30]:
print("TABLA COMPARATIVA DE RESULTADOS")

# Mostrar tabla resumen detallada
for idx, row in df_resultados.iterrows():
    print(f"\n{'='*80}")
    print(f"QUERY {idx+1}: {row['Query']}")
    print(f"{'='*80}")
    print(f"TF-IDF Top-3: {row['TFIDF_top3']}")
    print(f"BM25 Top-3  : {row['BM25_top3']}")
    print(f"FAISS Top-3 : {row['FAISS_top3']}")
    print(f"\nDocumentos en común (top-10):")
    print(f"  • TF-IDF & BM25      : {row['Comunes_TFIDF_BM25']}/10")
    print(f"  • TF-IDF & FAISS     : {row['Comunes_TFIDF_FAISS']}/10")
    print(f"  • BM25 & FAISS       : {row['Comunes_BM25_FAISS']}/10")
    print(f"\nRAG:")
    print(f"  • Documentos usados  : {row['Docs_RAG']}")
    print(f"  • Coincide con TF-IDF: {row['RAG_en_TFIDF']}/3")
    print(f"  • Coincide con BM25  : {row['RAG_en_BM25']}/3")
    print(f"  • Coincide con FAISS : {row['RAG_en_FAISS']}/3")


TABLA COMPARATIVA DE RESULTADOS

QUERY 1: bayesian inference in astrophysics
TF-IDF Top-3: ['0708.1362', '0709.1504', '0706.1767']
BM25 Top-3  : ['0707.0930', '0710.0845', '0707.3969']
FAISS Top-3 : ['0707.1611', '0707.3962', '0709.0970']

Documentos en común (top-10):
  • TF-IDF & BM25      : 4/10
  • TF-IDF & FAISS     : 1/10
  • BM25 & FAISS       : 1/10

RAG:
  • Documentos usados  : ['0707.1611', '0707.3962', '0709.0970']
  • Coincide con TF-IDF: 0/3
  • Coincide con BM25  : 0/3
  • Coincide con FAISS : 3/3


## 11. ANÁLISIS COMPARATIVO ENTRE MODELOS

In [32]:
print("ANÁLISIS COMPARATIVO ENTRE MODELOS")

# Calcular estadísticas generales
total_queries = len(df_resultados)
promedio_comunes_tfidf_bm25 = df_resultados['Comunes_TFIDF_BM25'].mean()
promedio_comunes_tfidf_faiss = df_resultados['Comunes_TFIDF_FAISS'].mean()
promedio_comunes_bm25_faiss = df_resultados['Comunes_BM25_FAISS'].mean()
promedio_comunes_todos = df_resultados['Comunes_todos'].mean()

print(f"ESTADÍSTICAS GENERALES ({total_queries} consultas):")
print(f"{'─'*50}")
print(f"Promedio de documentos en común (top-10):")
print(f"  • TF-IDF & BM25      : {promedio_comunes_tfidf_bm25:.2f}/10 ({promedio_comunes_tfidf_bm25*10:.1f}%)")
print(f"  • TF-IDF & FAISS     : {promedio_comunes_tfidf_faiss:.2f}/10 ({promedio_comunes_tfidf_faiss*10:.1f}%)")
print(f"  • BM25 & FAISS       : {promedio_comunes_bm25_faiss:.2f}/10 ({promedio_comunes_bm25_faiss*10:.1f}%)")

# Análisis de similitud entre rankings
print(f"\nANÁLISIS DE SIMILITUD ENTRE RANKINGS:")
print(f"{'─'*50}")
print("• TF-IDF y BM25 tienen mayor similitud (ambos basados en términos)")
print("• FAISS usa embeddings semánticos, encuentra documentos conceptualmente similares")
print("• Las diferencias en ordenamiento reflejan estrategias de scoring distintas")
print("• FAISS puede encontrar documentos relevantes que los otros modelos no detectan")

# Análisis del módulo RAG
print(f"\nANÁLISIS DEL MÓDULO RAG:")
print(f"{'─'*50}")
total_rag_docs = sum([len(row['Docs_RAG']) for _, row in df_resultados.iterrows()])
total_rag_en_tfidf = sum(df_resultados['RAG_en_TFIDF'])
total_rag_en_bm25 = sum(df_resultados['RAG_en_BM25'])
total_rag_en_faiss = sum(df_resultados['RAG_en_FAISS'])

print(f"• Total documentos usados en RAG: {total_rag_docs}")
print(f"• Documentos RAG que aparecen en FAISS top-10: {total_rag_en_faiss}/{total_rag_docs} ({total_rag_en_faiss/total_rag_docs*100:.1f}%)")
print("• El RAG utiliza FAISS, por lo que naturalmente tiene alta coincidencia con este modelo")

ANÁLISIS COMPARATIVO ENTRE MODELOS
ESTADÍSTICAS GENERALES (1 consultas):
──────────────────────────────────────────────────
Promedio de documentos en común (top-10):
  • TF-IDF & BM25      : 4.00/10 (40.0%)
  • TF-IDF & FAISS     : 1.00/10 (10.0%)
  • BM25 & FAISS       : 1.00/10 (10.0%)

ANÁLISIS DE SIMILITUD ENTRE RANKINGS:
──────────────────────────────────────────────────
• TF-IDF y BM25 tienen mayor similitud (ambos basados en términos)
• FAISS usa embeddings semánticos, encuentra documentos conceptualmente similares
• Las diferencias en ordenamiento reflejan estrategias de scoring distintas
• FAISS puede encontrar documentos relevantes que los otros modelos no detectan

ANÁLISIS DEL MÓDULO RAG:
──────────────────────────────────────────────────
• Total documentos usados en RAG: 3
• Documentos RAG que aparecen en FAISS top-10: 3/3 (100.0%)
• El RAG utiliza FAISS, por lo que naturalmente tiene alta coincidencia con este modelo


## 12. EJEMPLO ADICIONAL DETALLADO DE UNA CONSULTA CON RAG

In [34]:

# Tomar la primera consulta del TXT como ejemplo
ejemplo_query = queries[0]
print(f"CONSULTA EJEMPLO: '{ejemplo_query}'")
print("="*80)

# Mostrar documentos recuperados por cada modelo
print("DOCUMENTOS RECUPERADOS POR CADA MODELO (Top-3):")
print("─"*60)

# TF-IDF
print("TF-IDF:")
tfidf_ejemplo = search_tfidf(ejemplo_query, top_k=3)
for i, (_, doc) in enumerate(tfidf_ejemplo.iterrows(), 1):
    print(f"  {i}. [{doc['id']}] {doc['title'][:80]}...")

# BM25
print("\nBM25:")
bm25_ejemplo = search_bm25(ejemplo_query, top_k=3)
for i, (_, doc) in enumerate(bm25_ejemplo.iterrows(), 1):
    print(f"  {i}. [{doc['id']}] {doc['title'][:80]}...")

# FAISS
print("\nFAISS:")
faiss_ejemplo = search_faiss(ejemplo_query, top_k=3)
for i, (_, doc) in enumerate(faiss_ejemplo.iterrows(), 1):
    print(f"  {i}. [{doc['id']}] {doc['title'][:80]}...")

CONSULTA EJEMPLO: 'superconductivity in high temperature materials'
DOCUMENTOS RECUPERADOS POR CADA MODELO (Top-3):
────────────────────────────────────────────────────────────
TF-IDF:
  1. [0708.3142] Basic principle of superconductivity...
  2. [0708.1388] Superconductivity on the border of weak itinerant ferromagnetism in
  UCoGe...
  3. [0707.3660] Basic physics of solid materials...

BM25:
  1. [0708.1947] Vitaly Ginzburg and High Temperature Superconductivity: Personal
  Reminiscences...
  2. [0707.1020] Gutzwiller-RVB Theory of High Temperature Superconductivity: Results
  from Reno...
  3. [0706.0059] High-Tc superconductivity originated from strong spin-charge
  correlation: indi...

FAISS:
  1. [0706.0778] Nodeless Superconductivity in the Noncentrosymmetric Superconductor
  $Mg_{10}Ir...
  2. [0706.0059] High-Tc superconductivity originated from strong spin-charge
  correlation: indi...
  3. [0706.4398] Strong-Coupling Superconductivity of CeIrSi$_3$ with the
  Non-centrosym

In [35]:
# Generar y mostrar respuesta RAG
print("RESPUESTA GENERADA POR RAG:")
print("="*80)
respuesta_ejemplo, docs_ejemplo = generar_respuesta_rag(ejemplo_query) 
print(respuesta_ejemplo)

RESPUESTA GENERADA POR RAG:
Según el contexto proporcionado, la superconductividad a alta temperatura (High-Tc superconductivity) está relacionada con una dependencia lineal de la resistividad en función de la temperatura en un amplio rango. Esta característica se observa en los estados normales, cerca de los estados superconductores, en materiales como los superconductores basados en Cu y Fe (ID: 0706.0059).

Los cálculos teóricos sugieren que la superconductividad a alta temperatura podría ser inducida por una fuerte correlación espín-carga, ya que esta correlación es la que domina la resistividad y resulta en su dependencia lineal con la temperatura (ID: 0706.0059).


In [36]:
print("DOCUMENTOS UTILIZADOS POR RAG:")
print("="*80)
for i, doc in enumerate(docs_ejemplo, 1):
    print(f"{i}. ID: {doc['id']}")
    print(f"   Título: {doc['title']}")
    print(f"   Resumen: {doc['abstract'][:200]}...")

DOCUMENTOS UTILIZADOS POR RAG:
1. ID: 0706.0778
   Título: Nodeless Superconductivity in the Noncentrosymmetric Superconductor
  $Mg_{10}Ir_{19}B_{16}$
   Resumen:   We measured the resistivity, diamagnetization and low temperature specific
heat of the newly discovered noncentrosymmetric superconductor
$Mg_{10}Ir_{19}B_{16}$. It is found that the superconducting...
2. ID: 0706.0059
   Título: High-Tc superconductivity originated from strong spin-charge
  correlation: indication from linear temperature dependence of resistivity
   Resumen:   Both the highest- and the linear temperature dependence of the resistivity in
wide temperature range appear at the optimally doped regions of Cu-based
superconductors1,2,3,4,5, and the highest- of F...
3. ID: 0706.4398
   Título: Strong-Coupling Superconductivity of CeIrSi$_3$ with the
  Non-centrosymmetric Crystal Structure
   Resumen:   We studied the pressure-induced superconductor CeIrSi$_3$ with the
non-centrosymmetric tetragonal structure unde